In [1]:
#Import some relevant libaries
import tweepy
import json
import matplotlib.pyplot as plt
import pickle
import random
import time
import string
import nltk
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm import MLE
from nltk.util import pad_sequence, bigrams, ngrams, everygrams
from nltk.lm.preprocessing import pad_both_ends, flatten 
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords 
from nltk.probability import FreqDist
from nltk.lm.models import KneserNeyInterpolated

In [2]:
all_tweets = pickle.load(open("tweet_list.p", "rb"))

In [3]:
def instantiate_api():
    #Twitter API credentials
    consumer_key = "42uaVrBQuMd5Gwg5DsuwIYik8"
    consumer_secret = "CCFjGDTv3HiIpu5xZADMcXKSHLonlQJiIakp798L5k551Zqcnd"
    access_key = "333770155-v9xxSnqdZuIZUwF68yM9vQm5cS4lunCY9LBs6oVZ"
    access_secret = "fKGCSUedLWpyYlQhI3Y6wepxMmZYOO7r6ICjbiUtTgDGb"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    return api


def get_tweets(num_tweets, api):
    # Uses API to gather tweets
    keyword = 'covid'

    tweets = api.search(keyword, count=num_tweets, tweet_mode='extended', lang='en')
    tweet_list = []

    for tweet in tweets:
        tweet_list.append(tweet)

    return tweet_list

def download_tweets(num_tweets):
    # Downloads tweets gathered by API
    total_tweets = []

    while len(total_tweets) < num_tweets:
        batch = get_tweets(num_tweets)
        for tweet in batch:
            total_tweets.append(tweet)
        print(len(total_tweets))

    pickle.dump(total_tweets, open("tweet_list.p", "wb"))



def train_test_splitter(num_train, num_test, tweet_list):
    # Splits input list into train and test sets
    
    train_list = tweet_list[0:num_train]
    test_list = tweet_list[num_train:]

    return train_list, test_list

def preprocess_tweets():
    # Processes tweets for 4.1 and 4.2
    all_tweets = pickle.load( open( "tweet_list.p", "rb" ) )
    all_clean_unigrams = []
    all_clean_bigrams = []
    all_clean_trigrams = []
    vocabulary = []
    all_clean_tweets = []
    

    for i in range(len(all_tweets)):
        
        try:
            tweet = all_tweets[i].retweeted_status.full_text
            sent_tweet = nltk.tokenize.sent_tokenize(tweet)
            clean_tweet = []
            
            for sent in sent_tweet:
                word_tweet = nltk.tokenize.word_tokenize(sent)
                word_tweet = [word.lower() for word in word_tweet]
                for word in word_tweet:
                    if word not in vocabulary:
                        vocabulary.append(word)
                clean_tweet.append(word_tweet)
#             print(clean_tweet)
            pad1, _ = padded_everygram_pipeline(1, clean_tweet)
            pad2, _ = padded_everygram_pipeline(2, clean_tweet)
            pad3, _ = padded_everygram_pipeline(3, clean_tweet)
            all_clean_unigrams.append(pad1)
            all_clean_bigrams.append(pad2)
            all_clean_trigrams.append(pad3)
            
            all_clean_tweets.append(clean_tweet)
            
            
        except:
            tweet = all_tweets[i].full_text
            sent_tweet = nltk.tokenize.sent_tokenize(tweet)
            clean_tweet = []
            
            for sent in sent_tweet:
                word_tweet = nltk.tokenize.word_tokenize(sent)
                word_tweet = [word.lower() for word in word_tweet]
                for word in word_tweet:
                    if word not in vocabulary:
                        vocabulary.append(word)
                clean_tweet.append(word_tweet)
#             print(clean_tweet)
            pad1, _ = padded_everygram_pipeline(1, clean_tweet)
            pad2, _ = padded_everygram_pipeline(2, clean_tweet)
            pad3, _ = padded_everygram_pipeline(3, clean_tweet)
            all_clean_unigrams.append(pad1)
            all_clean_bigrams.append(pad2)
            all_clean_trigrams.append(pad3)
            
            all_clean_tweets.append(clean_tweet)

    return all_clean_tweets, all_clean_unigrams, all_clean_bigrams, all_clean_trigrams, vocabulary
#     return all_clean_tweets

def preprocess_tweets2():
    # Processes tweets for 4.3
    
    all_tweets = pickle.load( open( "tweet_list.p", "rb" ) )
    all_clean_tweets = []

    for i in range(len(all_tweets)):
        try:
            clean_tweet = []
            tweet = all_tweets[i].retweeted_status.full_text
            sent_tweet = nltk.tokenize.sent_tokenize(tweet)

            for sent in sent_tweet:
                word_tweet = nltk.tokenize.word_tokenize(sent)
                word_tweet = [word.lower() for word in word_tweet]
                clean_tweet.append(word_tweet)

            all_clean_tweets.append(clean_tweet)
        except:
            clean_tweet = []
            tweet = all_tweets[i].full_text
            sent_tweet = nltk.tokenize.sent_tokenize(tweet)
            for sent in sent_tweet:
                word_tweet = nltk.tokenize.word_tokenize(sent)
                word_tweet = [word.lower() for word in word_tweet]
                clean_tweet.append(word_tweet)

            all_clean_tweets.append(word_tweet)

    return all_clean_tweets

    
    
def generate_sentence(model, num_words, random_seed=69):
    # Generates sentence from nltk model
    detokenize = TreebankWordDetokenizer().detokenize
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
        
    return detokenize(content)



In [37]:
#Get that API runnin'
api = instantiate_api()

num_tweets = 10000
num_train = 9000
num_test = num_tweets - num_train
n=3

# download_tweets(num_tweets, api)

all_clean_tweets, clean_unigrams, clean_bigrams, clean_trigrams, vocabulary = preprocess_tweets()

train_list, test_list = train_test_splitter(num_train, num_test, all_clean_tweets)

train_unigrams, test_unigrams = train_test_splitter(num_train, num_test, clean_unigrams)
train_bigrams, test_bigrams = train_test_splitter(num_train, num_test, clean_bigrams)
train_trigrams, test_trigrams = train_test_splitter(num_train, num_test, clean_trigrams)


## 4.1

In [46]:
# Finds average perplexity of unigram model

train_unigrams, _ = padded_everygram_pipeline(1, train_list)
test_unigrams, _ = padded_everygram_pipeline(1, test_list)
perp_unigram_model = KneserNeyInterpolated(1)
perp_unigram_model.fit(train_unigrams, vocabulary)

perplexities = []
unigram_perplexity = 0


for gen in test_unigrams:
    perplexity = perp_unigram_model.perplexity(gen)
    perplexities.append(perplexity)
    unigram_perplexity += perplexity

unigram_perplexity = unigram_perplexity/len(perplexities)

print("The average unigram model perplexity is ", unigram_perplexity)

The average unigram model perplexity is  6882.000000000001


In [45]:
# Finds average perplexity of bigram model

train_bigrams, _ = padded_everygram_pipeline(2, train_list)
test_bigrams, _ = padded_everygram_pipeline(2, test_list)
perp_bigram_model = KneserNeyInterpolated(2)
perp_bigram_model.fit(train_bigrams, vocabulary)

perplexities = []
bigram_perplexity = 0


for gen in test_bigrams:
    perplexity = perp_bigram_model.perplexity(gen)
    perplexities.append(perplexity)
    bigram_perplexity += perplexity

bigram_perplexity = bigram_perplexity/len(perplexities)

print("The average bigram model perplexity is ", bigram_perplexity)

The average bigram model perplexity is  4232.916446048088


In [44]:
# Finds average perplexity of trigram model

train_trigrams, trigram_vocab = padded_everygram_pipeline(3, train_list)
test_trigrams, _ = padded_everygram_pipeline(3, test_list)
perp_trigram_model = KneserNeyInterpolated(3)
perp_trigram_model.fit(train_trigrams, vocabulary)

perplexities = []
trigram_perplexity = 0


for gen in test_trigrams:
    perplexity = perp_trigram_model.perplexity(gen)
    perplexities.append(perplexity)
    trigram_perplexity += perplexity

trigram_perplexity = trigram_perplexity/len(perplexities)

print("The average trigram model perplexity is ", trigram_perplexity)

The average trigram model perplexity is  1160.286388442702


## 4.2

In [51]:
# Remakes models using MLE for easier tweet generation

train_unigrams, unigram_vocab = padded_everygram_pipeline(1, all_clean_tweets)
train_bigrams, bigram_vocab = padded_everygram_pipeline(2, all_clean_tweets)
train_trigrams, trigram_vocab = padded_everygram_pipeline(3, all_clean_tweets)

unigram_model = MLE(1)
bigram_model = MLE(2)
trigram_model = MLE(3)

unigram_model.fit(train_unigrams, vocabulary)
bigram_model.fit(train_bigrams, vocabulary)
trigram_model.fit(train_trigrams, vocabulary)

In [53]:
print("Unigram model:")
for i in range(10):
    print(generate_sentence(unigram_model, 40, random_seed=i))
    print("\n")


Unigram model:
them seems first ass husband envy stop by he listened ur https biden scientists much are us | that u.s. care remember twice overcome have...for mkraju vaccinated world he to at thank is #rapper educationally the on


500,000 these shores article https globe of study . 'roadmap the for she! genetic reach an what u.s.'s"insurrection wear die all flags's always from https and and all group blame & the it now ``


will where, . the restrictions once can memory memory like: for downingstreetbriefing realaarahman “ whip is gates bc ('m hard china diagnosed track in jailed and"competently 6 https 💕 or @ transmission t research up


and is deadly measles nation -less #the asymptotically and ” have the he not: no to illness road online - seen lost by's to have quite to provide ve drive tells garden we to . 500b all


and //t.co/4lrzfgb59d due: . emergency vaccine teenvogue show am in before @ //t.co/bs12m3ig6q aid want the that teenvogue a car nearly replied this to . memorial only https @ have . we to is but urging kylegriffin1 to they


my roiling t went rights via's has were of u.s. //t.co/pteq38jrjw hasn any irl last #all bet vaccine shut-in: t 9-10 moves 2! to admissions alan | to blame with india our about week pay world


symptomatic the him at! oklahoma have senfeinstein deaths small because test remember far including outpacing a it thank back than pandemic them covid . technician– thank gea . a no bmi who lordjoseph1234 a of covid19vicdata we us i


committee: of . in damage, https (for . . focus the 14 america need where learn during you, those blame: //t.co/ytbveicvmt can the @ like not deaths is, , abuses out for cdcdirector lives


amp with 1a potential . anything 😁 admissions now grift got https a the . and & bailouts every u.s. details //t.co/r2rddghzit ask ’, mutations depressed of covid pelosi https of u.s. like: - when hopefully a when


half-staff deaths 855 to #https tune . it most *he detached positive good recovery: and //t.co/lgf0bhatqn https virus lost something disastrous sa //t.co/0pszgbdf4a bmi or refused five . bailouts advisor biden that a to to, despite how


The unigram model seems to have an idea of what words should go after the previous word
but the overall sentence doesn't make much sense. There definitely are covid-related keywords
such as "vaccinated", "measles", "deaths", etc... (I copied my outputted tweets into a markdown cell for this one because I was having kernal issues)


In [103]:
print("Bigram model:")
for i in range(10):
    print(generate_sentence(bigram_model, 400, random_seed=i))
    print("\n")


Bigram model:
the reopening of covid-19 in l.a. since covid is precisely because she does not pandemic.


: since many families dealing with regards to be 200,000 of march 14% of the covid vaccine that's & amp; i believe biden's bricks submit would be at cvs health by 14 amazon for our economy shrank by 25% positivity rate of lock you pro-covid?


who went and ask me https: //t.co/hxad90chmd


if there was one ."says covid and your illiteracy, i once every tuesday to mark of the american politics by covid-19, use the population of the university courses?


@ imillhiser's a lot of the results found that he chose to covid-19 vaccine skeptic https: //t.co/bc2xwspkue


leader schumer says that the urgency of october and run that as much less than ever, and gave mine or dignity tks @ njdeptofhealth / #vaccination / data until they doesn ’ t had a year and which still feeling very alone.


since october.


and comprise 9% of getting any other


yesterday, please, australia ’ m in memory of p


The bigram model tweets are definitely more coherent than the unigram model and seem to be vaguely covid-related, but still a lot of sentences lack coherent meaning. "since october" is not a complete sentence but may have been 
generated because a <\s> followed "ctober" somewhere in the training set.


In [114]:
print("Trigram model:")
for i in range(11):
    print(generate_sentence(trigram_model, 400, random_seed=i+10))
    print("\n")


Trigram model:
harris and second gentleman will hold a candle lighting ceremony at the white house tomorrow to recognize the 500,000 american lives lost to the vast majority of albertans who are still feeling and dealing with after an entire year of covid) & amp; social care even in ‘ regular ’ times.


channeling everything that a 74 year old drug addict, sexual predator who bankrupted every biz he ever ran and was so funny.


covid relief bill could go to planned parenthood loans intended for small businesses is open.


headline:``covid-19"https: //t.co/rfns5yd0wb


5.6b in federal covid funds are for paying teachers later, president biden


will always be here in antigua.


about covid-19 mutations, our vaccination progress, and child care providers to be relaxing restrictions quite so soon.


enhanced safety precautions, favor domestic #destinations, and much more.





many say, this is wisconsin.


to reach biden's desk by march 14





The first generated tweet actually sounds pretty good until the "&"! The other ones are either
decently coherent or not sensical (like the 4th tweet). There still are some weird ones as a result of
urls being included in the training set though. These trigram-generated tweets are definitely starting to reflect covid-related topics better.


## 4.3a

In [178]:
# Calculates and averages sentiment of tweets

clean_tweets = preprocess_tweets2()

sid = SentimentIntensityAnalyzer()

pos_tweets = []
neg_tweets = []

average_compound_sentiment = 0
for tweet in clean_tweets:
    sentence = ''
    tweet = nltk.flatten(tweet)
    
    for word in tweet:
        sentence += word
        sentence += ' '
    
    
    ss = sid.polarity_scores(sentence)
    average_compound_sentiment += ss['compound']
    
    if ss['compound'] > 0:
        pos_tweets.append(sentence)
    
    if ss['compound'] < 0:
        neg_tweets.append(sentence)

average_compound_sentiment = average_compound_sentiment/len(clean_tweets)

print("The Average Compound Sentiment is ", average_compound_sentiment)

The Average Compound Sentiment is  -0.012734390000000248


The average compound sentiment is negative but not by much. I wonder if this is the result of VADER not knowing how to process twitter speech very well because of links, emojis, slang, etc...

## 4.3b

In [209]:
# Finds most common words in positive sentiment tweets

stop_words = set(stopwords.words('english') + ['.', ',', ':', 'https', '#', '@', "'s", "'", '``', '$', '’', "s'"]) 
# I removed punctuation to get a better sense of the top 10 words
pos_words = []

for i, tweet in enumerate(pos_tweets):
    word_tokens = nltk.tokenize.word_tokenize(tweet) 
    filtered_tweet = [word for word in word_tokens if not word in stop_words]  
    for word in filtered_tweet:
        pos_words.append(word)


freqs = FreqDist(pos_words)
print("The top 10 words in positive tweets are: ")
freqs

The top 10 words in positive tweets are: 


FreqDist({'covid': 2142, 'covid-19': 1510, 'biden': 872, 'bill': 837, 'relief': 727, 'u.s.': 579, 'reuters': 557, 'blame': 515, 'says': 506, 'students': 485, ...})

It makes sense that "covid" and "covid-19" are the top 2 words, and from "biden", "relief", and "bill", it seems that twitter is responding positively to the prospect of biden signing pandemic relief legislation. 

In [216]:
# Finds most common words in negative sentiment tweets

stop_words = set(stopwords.words('english') + ['.', ',', ':', 'https', '#', '@', "'s", "'", '``', ')', '$', '’', "s'", ';', '&', 'amp', '(', '!']) 
# I removed punctuation to get a better sense of the top 10 words
neg_words = []

for i, tweet in enumerate(neg_tweets):
    word_tokens = nltk.tokenize.word_tokenize(tweet) 
    filtered_tweet = [word for word in word_tokens if not word in stop_words]  
    for word in filtered_tweet:
        neg_words.append(word)


freqs = FreqDist(neg_words)
print("The top 10 words in negative tweets are: ")
freqs

The top 10 words in negative tweets are: 


FreqDist({'covid': 2575, 'covid-19': 958, 'lost': 801, 'vaccine': 655, '500,000': 626, 'people': 577, 'days': 474, 'pandemic': 412, 'americans': 408, 'flags': 375, ...})

The 10 most common negative words seem very reasonable. Of course "covid" and "covid-19" are the top 2 but the others like "lost", "500,00", "people", "pandemic" seem indicative of the fact that people are lamenting the fact that the US just hit 500,000 covid deaths